In [15]:
import os
import pandas as pd
import openai
from dotenv import load_dotenv
import time

In [16]:
load_dotenv()

True

In [17]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [18]:
articles = pd.read_excel('sample_fine_tune.xlsx')

In [19]:
articles

,Article Info,prompt,completion
0,Date Published: 2023-02-01T18:00:29Z\nTitle: S...,Date Published: 2023-02-01T18:00:29Z\nContent:...,NaN
1,Date Published: 2023-01-28T17:42:04Z\nTitle: U...,Date Published: 2023-01-28T17:42:04Z\nContent:...,NaN
2,Date Published: 2023-01-24T22:25:00Z\nTitle: N...,Date Published: 2023-01-24T22:25:00Z\nContent:...,NaN
3,Date Published: 2023-01-24T14:06:00Z\nTitle: S...,Date Published: 2023-01-24T14:06:00Z\nContent:...,NaN
4,Date Published: 2023-01-31T20:52:45Z\nTitle: C...,Date Published: 2023-01-31T20:52:45Z\nContent:...,NaN
...,...,...,...
95,Date Published: 2023-02-09T14:55:00Z\nTitle: B...,Date Published: 2023-02-09T14:55:00Z\nContent:...,NaN
96,Date Published: 2023-02-14T21:13:00Z\nTitle: S...,Date Published: 2023-02-14T21:13:00Z\nContent:...,NaN
97,Date Published: 2023-02-12T15:00:59Z\nTitle: I...,Date Published: 2023-02-12T15:00:59Z\nContent:...,NaN
98,Date Published: 2023-01-27T14:45:00Z\nTitle: W...,Date Published: 2023-01-27T14:45:00Z\nContent:...,NaN


In [29]:
# wrapper for rate limit exponential delay
def ratelimiterror_exponential_backoff(
    func,
    init_delay=60,
    expo_base=2,
    errors=(openai.error.RateLimitError),
    max_retries=5
):
    def wrapper(*args, **kwargs):
        retries = 0
        delay = init_delay
        while True:
            try:
                return func(*args, **kwargs)
            except errors as e:
                retries += 1
                if retries > max_retries:
                    raise Exception(f'Exceeded maximum number of retries ({max_retries})')
                    
                print(f'RateLimitError: Sleeping for {delay} seconds')
                time.sleep(delay)
                delay *= expo_base
                print(f'Next error wait time is {delay} seconds. {max_retries - retries} retries remaining')
                
                
            except Exception as e:
                raise e
    return wrapper
                

In [47]:
@ratelimiterror_exponential_backoff
def generate_questions(prompt):
    prompt = trim_text(prompt)
    wait_time = 3.5
    response = openai.Completion.create(
        model='text-davinci-003',
        prompt=f'Write questions based on the article below\nArticle: {prompt}\nQuestions:\n1.',
        presence_penalty=0.5,
        frequency_penalty=0.5,
        temperature=1,
        top_p=1,
        n=3
    )
    time.sleep(wait_time)
        
    questions = '1.' + response['choices'][0]['text']
    #print(questions)
    return questions

@ratelimiterror_exponential_backoff
def generate_answers(context, questions):
    context = trim_text(context)
    wait_time = 3.5
    response = openai.Completion.create(
        model='text-davinci-003',
        prompt=f'Write answers to the questions based on the article below\nQuestions:\n{questions}\n\nArticle: {context}\n\nAnswers:\n1.',
        presence_penalty=0.5,
        frequency_penalty=0.5,
        temperature=1,
        top_p=1,
        stop=['\n\n']
    )
    time.sleep(wait_time)
    
    answers = '1.' + response['choices'][0]['text']
    return '1.' + answers

def trim_text(text):
    while len(text) > 6500:
        text = text[:text.rfind('\n')]
    return text

def generate_prompt(context, questions):
    return f'Write answers to the questions based on the article below\nQuestions:\n{questions}\n\nArticle: {context}\n\nAnswers:\n1.'


In [46]:
articles['questions'] = articles['prompt'].apply(generate_questions)

In [60]:
context = articles['prompt'][0]
wait_time = 3.5
response = openai.Completion.create(
    model='text-davinci-003',
    prompt=f'Write three questions based on the article below\nArticle: {context}\nQuestions:\n1.',
    presence_penalty=0.5,
    frequency_penalty=0.5,
    temperature=1,
    top_p=1,
    n=3
)
time.sleep(wait_time)
print(response)

{
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " What kind of processor does the Galaxy Book 3 Ultra feature?\n2. How"
    },
    {
      "finish_reason": "length",
      "index": 1,
      "logprobs": null,
      "text": " What specific specifications does the Samsung Galaxy Book 3 Ultra have?\n2. What"
    },
    {
      "finish_reason": "length",
      "index": 2,
      "logprobs": null,
      "text": " What features does the Galaxy Book 3 Ultra have that makes it stand out from competition"
    }
  ],
  "created": 1676419900,
  "id": "cmpl-6jzgaVBtLIS2uDZKp1UMOiK5JJUYq",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 48,
    "prompt_tokens": 1019,
    "total_tokens": 1067
  }
}


In [61]:
len(context)

4202

In [51]:
articles['questions'][1]

"1. What agreement have the US, Netherlands and Japan reportedly reached to restrict China's access"

{
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " What components make up the Samsung Galaxy Book 3 lineup?\n2. How do"
    }
  ],
  "created": 1676419801,
  "id": "cmpl-6jzezXy6jIszxLSgKMqZ8xXfFfIGE",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 1019,
    "total_tokens": 1035
  }
}


In [42]:
articles.iloc[70]['prompt']

'Date Published: 2023-01-18T18:48:18Z\nContent:\n\nDeep learning is a field with intense computational requirements, and your choice of GPU will fundamentally determine your deep learning experience. But what features are important if you want to buy a new GPU? GPU RAM, cores, tensor cores, caches? How to make a cost-efficient choice? This blog post will delve into these questions, tackle common misconceptions, give you an intuitive understanding of how to think about GPUs, and will lend you advice, which will help you to make a choice that is right for you.\n\nThis blog post is designed to give you different levels of understanding of GPUs and the new Ampere series GPUs from NVIDIA. You have the choice: (1) If you are not interested in the details of how GPUs work, what makes a GPU fast compared to a CPU, and what is unique about the new NVIDIA RTX 40 Ampere series, you can skip right to the performance and performance per dollar charts and the recommendation section. The cost/perform